In [1]:
from disqco.circuits.cp_fraction import cp_fraction, cz_fraction
from disqco.graphs.GCP_hypergraph import QuantumCircuitHyperGraph
from disqco.graphs.hypergraph_methods import calculate_full_cost_hetero
from qiskit import transpile
from disqco.parti.FM.FM_methods import set_initial_partitions
from disqco.circuits.QAOA import QAOA_random
from qiskit.circuit.library import QFT, QuantumVolume
import pickle
from copy import deepcopy
from disqco.graphs.quantum_network import *
from disqco.parti.FM.multilevel_FM import MLFM_recursive_hetero
import networkx as nx
from copy import deepcopy
import numpy as np
from disqco.graphs.quantum_network import QuantumNetwork
import os
import json

num_qubits_list = [128,256,512,1024]
network_types = ['linear', 'grid']
num_partitions_and_coarsening_factors = [(4,2), (8,2), (16,2), (32,2), (64,2), (8,4), (16,4), (32,4), (64,4),(16,8),(64,8)]

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_COARENING_POST_PROCESSED.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

In [ ]:
from disqco.parti.FM.net_coarsened_FM import run_net_coarsened_FM
from disqco.graphs.quantum_network import linear_coupling, grid_coupling
import time
iterations = 5 
for num_qubits in num_qubits_list:
    for network_type in network_types:
        for num_partitions_and_coarsening_factor in num_partitions_and_coarsening_factors:
            for iteration in range(iterations):
                num_partitions, coarsening_factor = num_partitions_and_coarsening_factor
                print(f"num_qubits: {num_qubits}, network_type: {network_type}, num_partitions: {num_partitions}, coarsening_factor: {coarsening_factor}")
                level_limit = 8 
                seed = np.random.randint(0, 10000)
                circuit = cp_fraction(num_qubits=num_qubits,
                                        depth=num_qubits,
                                        fraction=0.5,
                                        seed=seed)
                
                circuit = transpile(circuit, basis_gates=['cp', 'u'])

                qpu_sizes = [int(np.ceil(num_qubits / num_partitions)) + 1 for i in range(num_partitions)]

                if network_type == 'linear':
                    coupling = linear_coupling(num_partitions)
                elif network_type == 'grid':
                    coupling = grid_coupling(num_partitions)

                initial_qpu_sizes = {i: qpu_sizes[i] for i in range(num_partitions)}
                initial_network = QuantumNetwork(qpu_sizes, coupling)

                network = deepcopy(initial_network)
                initial_graph = QuantumCircuitHyperGraph(circuit, group_gates=True, anti_diag=True)
                depth = initial_graph.depth
                graph_no_grouping = QuantumCircuitHyperGraph(circuit, group_gates=False, anti_diag=True)

                graph = deepcopy(initial_graph)
                assignment = set_initial_partitions(network, num_qubits, depth)

                initial_cost_no_grouping = calculate_full_cost_hetero(hypergraph=graph_no_grouping, assignment=assignment, num_partitions=num_partitions, costs = {}, network=network)

                print("Initial cost with no grouping: ", initial_cost_no_grouping)

                initial_cost = calculate_full_cost_hetero(hypergraph=graph, assignment=assignment, num_partitions=num_partitions, costs = {}, network=network)

                print("Initial Cost: ", initial_cost)

                try:
                    start_time = time.time()
                    cost, final_assignment = run_net_coarsened_FM(graph, network, l=coarsening_factor, multiprocessing=True, level_limit=1, passes_per_level=10)
                    final_cost = calculate_full_cost_hetero(hypergraph=graph, assignment=final_assignment, num_partitions=num_partitions, costs = {}, network=network)
                    end_time = time.time()
                    results_entry = {
                        "num_qubits": num_qubits,
                        "network_type": network_type,
                        "num_partitions": num_partitions,
                        "coarsening_factor": coarsening_factor,
                        "level_limit": level_limit,
                        "initial_cost_no_grouping": initial_cost_no_grouping,
                        "initial_cost": initial_cost,
                        "final_cost": final_cost,
                        "iteration": iteration,
                        "time_taken": end_time - start_time,
                        "seed": seed
                    }
                    detailed_results.append(results_entry)


                except Exception as e:
                    print(f"Error occurred: {e}")
                    results_entry = {
                        "num_qubits": num_qubits,
                        "network_type": network_type,
                        "num_partitions": num_partitions,
                        "coarsening_factor": coarsening_factor,
                        "level_limit": level_limit,
                        "initial_cost_no_grouping": initial_cost_no_grouping,
                        "initial_cost": initial_cost,
                        "final_cost": None,
                        "iteration": iteration,
                        "time_taken": None,
                        "seed": seed
                    }
                    detailed_results.append(results_entry)
                    

                with open(detailed_filename, "w") as f:
                    json.dump(detailed_results, f, indent=2)


num_qubits: 128, network_type: linear, num_partitions: 4, coarsening_factor: 2
Number of layers: 130
Number of layers: 130
Initial cost with no grouping:  5000
Initial Cost:  3250
Time to coarsen graph: 0.27 seconds
Time for multilevel FM: 0.63 seconds
Time to coarsen graph: 0.23 seconds
Time to coarsen graph: 0.23 seconds
Time for multilevel FM: 0.27 seconds
Time for multilevel FM: 0.29 seconds
[33, 33, 33, 33]
All capacities are satisfied.
num_qubits: 128, network_type: linear, num_partitions: 4, coarsening_factor: 2
Number of layers: 130
Number of layers: 130
Initial cost with no grouping:  5226
Initial Cost:  3300
Time to coarsen graph: 0.24 seconds
Time for multilevel FM: 0.61 seconds
Time to coarsen graph: 0.23 seconds
Time to coarsen graph: 0.23 seconds
Time for multilevel FM: 0.27 seconds
Time for multilevel FM: 0.29 seconds
[33, 33, 33, 33]
All capacities are satisfied.
num_qubits: 128, network_type: linear, num_partitions: 4, coarsening_factor: 2
Number of layers: 130
Number 

KeyboardInterrupt: 